In [55]:
import os
import numpy as np
import pandas as pd
import re
from numpy import float32 as REAL
import pickle
from gensim import utils
import numpy as np
from collections import defaultdict
import ast
from scipy.spatial.distance import cdist as scipy_cdist
import matplotlib.pyplot as plt
import faiss
import tensorflow_hub as tf
import tensorflow_hub as hub
import time
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, classification_report
## GUSE Embeddings ##
guse_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
end_time = time.time()

In [56]:
#Preprocesing function
stop_words_list= ["a", "the" , "to" , "be" ,"is" , "an", "hi", "hello", "good morning","goodmorning",'goodevening',
                  'good evening', "hey", "wow", "buyer", "seller", "trader", 'good', 'noon','morning','evening', 'here',
                  "here recordingenjoy", "all", "everyone", "any", "help", "pls", "please", "certainely", "fee","like" ,"comment", "comment",
                 "friends","friend", "wi", "only", "verify", "asap", "some", "me", "fuck", "lol", "anyone", "cmnt", "omg", "wow","wtf", "oo",
                 "everyone", "page", "huh", "any", "new", "have", "my", "u", "attend", "look", "better", "interest"]

def lemmatize_word(text): 
    from nltk.tokenize import word_tokenize 
    from nltk.stem import WordNetLemmatizer
    lemmatizer=WordNetLemmatizer()
    word_tokens = word_tokenize(text) 
    lemmas = [lemmatizer.lemmatize(word,pos='v') for word in word_tokens] 
    return lemmas

def clean_text(text,stop_words=None):
    text = text.lower().strip() # Convert text to lower case and strip leading/trailing white spaces
    text = text.lower().strip() # Convert text to lower case and strip leading/trailing white spaces
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+', '', text) # Remove websites
    text = re.sub(r'[^\w\s$]', '', text) # Remove punctuation except $
    text= re.sub(r"cant",'can not',text)
    text= re.sub(r"didnt","did not",text)
    text= re.sub(r"dont","do not",text)
    text= re.sub(r"wasnt","was not",text)
    text= re.sub(r"wont","will not",text)    
    text= re.sub('\S+@\S+','',text) #removing emailadderess
    text= re.sub(r'https?:\/\/.*[\r\n]*', ' ', text) # removing all urls
    text= re.sub(r"\[.*?\]", " ",text)# removing data in square brackets
    text= re.sub("[^a-z\s$]+"," ",text)
    text= re.sub("thank you|thank you soo much|thank you so much|thanks a lot|thank you very much","thanks",text)
    text= re.sub(r"(\.|,|!)([a-zA-Z0-9]{1})",r"\1 \2",text)
    text= re.sub(' +',' ',text)  # removing extra spaces
    text= text.replace(",", "")
    text= text.replace(".", "")
    text= text.replace("?", "")
    text= text.replace("\n"," ")
    text= text.replace("'pm"," contact")# expansion contact
    text= text.replace("'call me"," contact")# expansion contact
    text= text.replace("'m"," am")# expansion am
    text= text.replace("'ll"," will")# expansion will
    text= text.replace("'d"," would")# expansion would
    text= text.replace("n't"," not")# expansion not
    text= text.replace("'ve"," have")# expansion have
    text= text.replace("'s"," is")# expansion is
    text= text.replace("'re"," are")# expansion are
    text= text.replace("'prize"," price")# expansion am
    text= text.replace("ain't","are not")
    text= text.replace("aint","are not")
    text= text.replace("cause","because")
    text= text.replace("couldn't","could not")
    text= text.replace("couldnt","could not")
    text= text.replace("didn't","did not")
    text= text.replace("didnt","did not")
    text= text.replace("doesn't","does not")
    text= text.replace("doesnt","does not")
    text= text.replace("don't","do not")
    text= text.replace("dont","do not")
    text= text.replace("everything's","everything is")
    text= text.replace("everythings","everything is")
    text= text.replace("everyones","everyone is")
    text= text.replace("everyone's","everyone is")
    text= text.replace("haven't","have not")
    text= text.replace("havent","have not")
    text= text.replace("hasn't","has not")
    text= text.replace("hasnt","has not")
    text= text.replace("hadn't","had not")
    text= text.replace("hadnt","had not")
    text= text.replace("he's","he is")
    text= text.replace("hes","he is")
    text= text.replace("i'll","i will")
    text= text.replace("ill","i will")
    text= text.replace("i'm","i am")
    text= text.replace("im","i am")
    text= text.replace("it's","it is")
    text= text.replace("its","it is")
    text= text.replace("i've","i have")
    text= text.replace("ive","i have")
    text= text.replace("let's","let us")
    text= text.replace("lets","let us")
    text= text.replace("she's","she is")
    text= text.replace("shes","she is")
    text= text.replace("shouldn't","should not")
    text= text.replace("shouldnt","should not")
    text= text.replace("that's","that is")
    text= text.replace("thats","that is")
    text= text.replace("there's","there is")
    text= text.replace("theres","there is")
    text= text.replace("they're","they are")
    text= text.replace("theyre","they are")
    text= text.replace("they've","they have")
    text= text.replace("theyve","they have")
    text= text.replace("wasn't","was not")
    text= text.replace("wasnt","was not")
    text= text.replace("we'd","we would")
    text= text.replace("wed","we would")
    text= text.replace("we'll","we will")
    text= text.replace("well","we will")
    text= text.replace("we're","we are")
    text= text.replace("were","we are")
    text= text.replace("weren't", "were not")
    text= text.replace("werent", "were not")
    text= text.replace("what's", "what is")
    text= text.replace("whats", "what is")
    text= text.replace("who's", "who is")
    text= text.replace("whos", "who is")
    text= text.replace("won't", "will not")
    text= text.replace("wont", "will not")
    text= text.replace("wouldn't", "would not")
    text= text.replace("wouldnt", "would not")
    text= text.replace("you'll", "you will")
    text= text.replace("youll", "you will")
    text= text.replace("you're", "you are")
    text= text.replace("youre", "you are")
    text= text.replace("you've", "you have")
    text= text.replace("youve", "you have")
    text= text.replace("kiss", "case")
    text= text.replace("gonna", "going to")
    text= text.replace("yeah", "yes")
    text= text.replace("may", "my")
    text= text.replace("ise", "is")
    text= text.strip()
    text= text.replace('uhhuh', '')
    text= lemmatize_word(text)
    if stop_words is None:
        stop_words= stop_words_list
    text = [word for word in text if word not in stop_words]
    return text
def remove_emoji(text):
    # Regular expression to match emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [58]:
#loading files
df = pd.read_csv(r'out.csv')
kpi_df = pd.read_excel(r'kpi_df.xlsx')

In [ ]:
#Applying preprocessing
df = df.drop_duplicates(subset = ['FeedText'],keep = 'last').reset_index(drop = True)
df=df[~(df.FeedText=='')]
df.dropna(subset=['FeedText'], inplace=True)
df['FeedText'] = df['FeedText'].apply(remove_emoji)
df['FeedText'] = df['FeedText'].str.replace('\d+', '')
df['FeedText'] = df.FeedText.dropna().apply(clean_text).map(lambda x: ' '.join(i for i in x))
df=df[~(df.FeedText=='')]

In [62]:
#Faiss Training and Simiarity Search Function
def faiss_index_creation(probing_baselines):
    dim = 512
    index = faiss.IndexFlatIP(dim)
    index.add(guse_embed((probing_baselines)).numpy())
    return index

def search_faiss(df,probing_baselines,faiss_index,faiss_similarity_threshold):
    li_res_guss = []
    query= df.FeedText.tolist()
    dist, idx = faiss_index.search((guse_embed(query).numpy()),2)
    for i in range(len(query)):
        if np.max(dist[i]) >= faiss_similarity_threshold:
            li_res_guss.append((query[i], idx[i][np.argmax(dist[i])], np.max(dist[i])))
        else:
            li_res_guss.append((query[i], None, 0))
    return li_res_guss
############################### Main Function For Checking Intent #########################################
def check_intent(df, kpi_df, kpi_type):
    #Provide threshold for different baselines
    faiss_similarity_threshold_buyer_seller = 0.448
    faiss_similarity_threshold_buyer= 0.447
    faiss_similarity_threshold_seller = 0.448
    #Loading the baselines for checking buyer, seller, buyer_seller
    d=defaultdict()
    kpi_df=kpi_df[kpi_df.sub_type==kpi_type]
    d['kpi_id']=kpi_df.id.iloc[0]
    d['kpi_name']=kpi_df.name.iloc[0]
    actor_type=kpi_df.actor_type.iloc[0]
    all_baselines=ast.literal_eval(kpi_df.processed_baseline.iloc[0])
    buyer_seller_baselines=all_baselines['buyer_seller']
    buyer_baselines=all_baselines['buyer_baselines']
    seller_baselines=all_baselines['seller_baselines']
    
    res=[]
    id = df['Id'].unique()[0]
    query=df.FeedText.tolist()
    if any(df.ProfileName.isna()):
        user_name="Name-Unavailable"
    else:
        try:
            user_name =  df['ProfileName'].unique()[0]
        except IndexError:
            user_name='None'
    #Creating indexes for similarity search
    index_buyer_seller=faiss_index_creation(buyer_seller_baselines)
    index_buyer=faiss_index_creation(buyer_baselines)
    index_seller=faiss_index_creation(seller_baselines)
    
    #Similarity Search for three baselines
    similarity_results_buyer_seller = search_faiss(df,buyer_seller_baselines,index_buyer_seller,faiss_similarity_threshold_buyer_seller)
    check_met_similarity_results_buyer_seller=[sm[2] for sm in similarity_results_buyer_seller if  sm[2]>0]
    
    similarity_results_buyer = search_faiss(df,buyer_baselines,index_buyer,faiss_similarity_threshold_buyer)
    check_met_similarity_results_buyer=[sm[2] for sm in similarity_results_buyer if  sm[2]>0]

    similarity_results_seller = search_faiss(df,seller_baselines,index_seller,faiss_similarity_threshold = faiss_similarity_threshold_seller)
    check_met_similarity_results_seller=[sm[2] for sm in similarity_results_seller if  sm[2]>0]
    
    #Main conditions check for intent classifications
    if len(check_met_similarity_results_buyer_seller)> 0:
        res.append((id, user_name, 'buyer_seller'))
    elif len(check_met_similarity_results_buyer) > 0:
        res.append((id, user_name, 'buyer'))
    elif len(check_met_similarity_results_seller) > 0:
        res.append((id, user_name, 'seller'))
    else:
        res.append((id, user_name, 'neutral'))
    #Storing the results
    d['score'] =  res
    return d

In [63]:
#Running the function on the whole dataframe
%%time
result = df.groupby('Id').apply(lambda x: check_intent(x, kpi_df , kpi_type="check_intent"))

CPU times: total: 2h 45min 47s
Wall time: 54min 39s


In [64]:
#Converting the results to dataframe
resultdf= pd.DataFrame(result)
resultdf.reset_index(inplace=True)
resultdf['user_name'] = [i['score'][0][1] if len(i['score'])>0 else 'NA' for i in resultdf[0]]
resultdf['intent'] = [i['score'][0][2] if len(i['score'])>0 else 'NA' for i in resultdf[0]]

In [65]:
resultdf

,Id,0,user_name,intent
0,5b1aa7ee8bb55b300895a774,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
1,5b1aa7ee8bb55b300895a776,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
2,5b1aa7ee8bb55b300895a778,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
3,5b1aa7ee8bb55b300895a77a,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Gian Fortuin,neutral
4,5b1aa7ee8bb55b300895a77c,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
...,...,...,...,...
134853,5d11b492a3df6323f881ab73,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Marwa Mohamed,neutral
134854,5d11b7e7a3df6323f881ab77,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Oun Sida,neutral
134855,5d11b7e7a3df6323f881ab7b,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Jonathan Reeves,neutral
134856,5d11b7e7a3df6323f881ab7f,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Michael Dass Sumaylo,neutral


In [66]:
resultdf['intent'].value_counts()

neutral         134453
buyer              251
buyer_seller       105
seller              49
Name: intent, dtype: int64

In [68]:
resultdf.to_excel('resultdf.xlsx')

In [82]:
# create a dictionary of your functions and variables
model = {'lemmatize_word': lemmatize_word,
         'clean_text': clean_text,
         'stop_words_list': stop_words_list,
         'faiss_index_creation' : faiss_index_creation,
         'search_faiss' : search_faiss,
         'check_intent' : check_intent}

# save the model as a pkl file
with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [80]:
import pickle

# load the model from the pkl file
with open('my_model.pkl', 'rb') as f:
    model = pickle.load(f)